In [1]:
#Import Dependencies
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import os
import time
from bs4 import BeautifulSoup
import re
import pandas as pd
import requests
from urllib.parse import urlparse
import json
import lxml.html

In [2]:
#Configure Chromedriver

chrome_install = ChromeDriverManager().install()

folder = os.path.dirname(chrome_install)
chromedriver_path = os.path.join(folder, "chromedriver.exe")

In [3]:
chrome_options = webdriver.ChromeOptions()
prefs = {"profile.default_content_setting_values.notifications" : 2}
chrome_options.add_experimental_option("prefs",prefs)
chrome_options.add_argument('--headless') # If you want to see the browser comment out this entire line

In [4]:
# Initialize Chrome WebDriver
browser = webdriver.Chrome(
    service = Service(chromedriver_path),
    options = chrome_options
)

In [5]:
with open('Config_file.json') as f:
    config = json.load(f)

In [6]:
#Setup search parameters
Username = config['facebook']['username']
Password = config['facebook']['password']

In [7]:
# Set up base URL
url = f'https://www.facebook.com/groups/ConsumerWatchdogBW'

# Visit the website
browser.get(url)

In [8]:
# Locate the button with aria-label="Decline optional cookies" (Europe)
try:
    phone_input = browser.find_element(By.XPATH, '//*[@id="login_popup_cta_form"]/div/div[3]/div/div/div/label')
    phone_input.send_keys(Username)
    password_input = browser.find_element(By.XPATH, '//*[@id="login_popup_cta_form"]/div/div[4]/div/div/div/label')
    password_input.send_keys(Password)
    login_click = browser.find_element(By.XPATH, '//*[@id="login_popup_cta_form"]/div/div[5]/div')
    login_click.click()
    print("Logged In!")
    
except:
    print("Could not Login!")
    pass

browser.implicitly_wait(20)

Logged In!


In [9]:
#Scroll down to load all results
try:
    # Get the initial scroll position
    last_height = browser.execute_script("return document.body.scrollHeight")
    time.sleep(10)
    html_list = []

    Count = 0
    
    while True:
        browser.implicitly_wait(15)
        # Scroll down to the bottom of the page using JavaScript
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(15)

        # Get the new scroll position
        new_height = browser.execute_script("return document.body.scrollHeight")

        # Retrieve the HTML
        html = browser.page_source

        # Parse the HTML using lxml
        root = lxml.html.fromstring(html)

        # Find all post elements
        post_elements = root.cssselect('div.x9f619.x1n2onr6.x1ja2u2z.x1jx94hy.x1qpq9i9.xdney7k.xu5ydu1.xt3gfkd.xh8yej3.x6ikm8r.x10wlt62.xquyuld')

        # Append the HTML of each post element to the list
        for post_element in post_elements:
            html_list.append(lxml.html.tostring(post_element).decode('utf-8'))
        
        Count += 1

        # Check if we've reached the bottom
        if new_height == last_height:
            break
        
        # Update the scroll position
        last_height = new_height
        
        print(f"scrolled: {Count}s")
        
except Exception as e:
    print(f"An error occurred: {e}")

scrolled: 1s
scrolled: 2s
scrolled: 3s
scrolled: 4s
scrolled: 5s
scrolled: 6s
scrolled: 7s
scrolled: 8s
scrolled: 9s
scrolled: 10s
scrolled: 11s
scrolled: 12s
scrolled: 13s
scrolled: 14s
scrolled: 15s
scrolled: 16s
scrolled: 17s
scrolled: 18s
scrolled: 19s
scrolled: 20s
scrolled: 21s
scrolled: 22s
scrolled: 23s
scrolled: 24s
scrolled: 25s
scrolled: 26s
scrolled: 27s
scrolled: 28s
scrolled: 29s
scrolled: 30s
scrolled: 31s
scrolled: 32s
scrolled: 33s
scrolled: 34s
scrolled: 35s
scrolled: 36s
scrolled: 37s
scrolled: 38s
scrolled: 39s
scrolled: 40s
scrolled: 41s
scrolled: 42s
scrolled: 43s
scrolled: 44s
scrolled: 45s
scrolled: 46s
scrolled: 47s
scrolled: 48s
scrolled: 49s
scrolled: 50s
scrolled: 51s
scrolled: 52s
scrolled: 53s
scrolled: 54s
scrolled: 55s
scrolled: 56s
scrolled: 57s
scrolled: 58s
scrolled: 59s
scrolled: 60s
scrolled: 61s
scrolled: 62s
scrolled: 63s
scrolled: 64s
scrolled: 65s
scrolled: 66s
scrolled: 67s
scrolled: 68s
scrolled: 69s
scrolled: 70s
scrolled: 71s
scrolled: 72s
s

In [10]:
# Combine the HTMLs
full_html = '\n'.join(html_list)

# Use BeautifulSoup to parse the HTML
soup = BeautifulSoup(full_html, 'html.parser')

#Close the browser
browser.close()

In [11]:
# Find all the posts' html
Posts = soup.find_all('div', class_='x9f619 x1n2onr6 x1ja2u2z x1jx94hy x1qpq9i9 xdney7k xu5ydu1 xt3gfkd xh8yej3 x6ikm8r x10wlt62 xquyuld')

In [12]:
count = 0

for entry in Posts:
    count += 1

print(count)
# print(Name.prettify())

4145


In [13]:
def parse_facebook_url(url):
  # Parse the URL
  parsed_url = 'https://www.facebook.com' + url.split('?')[0]
  return parsed_url

In [14]:
fb_Posts = []

for index, post in enumerate(Posts):
    PostID = index + 1
    Name = post.find('span', class_='html-span xdj266r x11i5rnm xat24cr x1mh8g0r xexx8yu x4uap5 x18d9i69 xkhd6sd x1hl2dhg x16tdsg8 x1vvkbs')
    if Name:
        Author = Name.get_text(strip=True)
    else:
        Author = None 
    Post_Text = post.find('div', class_='xdj266r x11i5rnm xat24cr x1mh8g0r x1vvkbs x126k92a')
    if Post_Text:
        Post_Content = Post_Text.get_text(strip=True)
    else:
        Post_Content = None 
    All_Reactions = post.find('span', class_='xt0b8zv x1jx94hy xrbpyxo xl423tq')
    if All_Reactions:
        No_of_Reacts = All_Reactions.get_text(strip=True)
    else:
        No_of_Reacts = None
    All_Comments = post.find('span', class_='x193iq5w xeuugli x13faqbe x1vvkbs x1xmvt09 x1lliihq x1s928wv xhkezso x1gmr53x x1cpjm7i x1fgarty x1943h6x xudqn12 x3x7a5m x6prxxf xvq8zen xo1l8bm xi81zsa')
    if All_Comments:
        No_of_Comments = All_Comments.get_text(strip=True)
    else:
        No_of_Comments = None

    TheComments = post.find_all('div', class_="xdj266r x11i5rnm xat24cr x1mh8g0r x1vvkbs")
    if TheComments:
        Comments = TheComments
    else:
        Comments = None

    if Comments is not None and len(Comments) > 0:
        Comment1 = Comments[0]
        First_Comment = Comment1.get_text()
        if len(Comments) > 1:
            Comment2 = Comments[1]
            Second_Comment = Comment2.get_text()
        else:
            Second_Comment = None
    else:
        First_Comment = None
        Second_Comment = None
    
    data = {
        # 'Post Id': PostID,
        'Author': Author,
        'Post Content': Post_Content,
        # 'Post Url': Post_Url
        'No of Reactions': No_of_Reacts,
        'No of Comments': No_of_Comments,
        'Comment One': First_Comment,
        'Comment Two': Second_Comment
    }
    fb_Posts.append(data)

Dataset = pd.DataFrame(fb_Posts)



In [15]:
Dataset

,Author,Post Content,No of Reactions,No of Comments,Comment One,Comment Two
0,None,None,None,None,None,None
1,MadamSure Mpho Thaolang,FNB BotswanaPilane branch..service ya lona e m...,16,8,Francistown branch ke maaka hela 72hrs still a...,"Yesterday was a disaster, ke heletse ke ya go ..."
2,Bongani Collin Junior Ngwenya,Botswana Telecommunications Corporation - BTCr...,38,10,I wouldn't advise you to download using di rou...,None
3,Leshman Kgangkenna,Multichoice Botswana Dstv..... Ke subscribe ka...,14,10,The answer i got from them ijhoooh,Heela nna kare a ke nna hela e seong
4,Kabelo Em Mbakile,Mpuelang le mascom they refusing to credit me ...,1,3,None,None
...,...,...,...,...,...,...
4140,George Cronje,Richard Harrimanare retailers allowed to enter...,14,15,No,FNB Botswana help us cause this is done at Cho...
4141,None,None,None,None,None,None
4142,None,None,None,None,None,None
4143,None,None,None,None,Consumer Watchdog (consumerwatchdog@bes.bw) is...,None


In [16]:
New_Data = Dataset.copy()

In [17]:
FB_Group_Data = New_Data.drop_duplicates(ignore_index=True)

In [18]:
FB_Group_Data = FB_Group_Data.dropna(subset=['Author'])

In [19]:
FB_Group_Data

,Author,Post Content,No of Reactions,No of Comments,Comment One,Comment Two
1,MadamSure Mpho Thaolang,FNB BotswanaPilane branch..service ya lona e m...,16,8,Francistown branch ke maaka hela 72hrs still a...,"Yesterday was a disaster, ke heletse ke ya go ..."
2,Bongani Collin Junior Ngwenya,Botswana Telecommunications Corporation - BTCr...,38,10,I wouldn't advise you to download using di rou...,None
3,Leshman Kgangkenna,Multichoice Botswana Dstv..... Ke subscribe ka...,14,10,The answer i got from them ijhoooh,Heela nna kare a ke nna hela e seong
4,Kabelo Em Mbakile,Mpuelang le mascom they refusing to credit me ...,1,3,None,None
5,Andrew Crab Sabokone,Mascom ako le baakanya internet ya lona every ...,10,21,I failed an online exam because of mascom. Le...,I give up ka internet ya Mascom
...,...,...,...,...,...,...
1166,Edwin Madome Serumola,None,22,13,"Dumelang Batswana betsho, batho re jelwa airti...","Dumelang Batswana betsho, batho re jelwa airti..."
1167,Eugene Lesedi,Le didimetse hela mme Orange ya re aketsa,4,6,E jelwe ke apps,Tobetsa *121# o cheke data ya ggo le ngongoreg...
1168,Cyrus Dracan,Botswana Telecommunications Corporation - BTCt...,7,18,Botswana Telecommunications Corporation - BTC ...,Mogoditshane is the same as tati siding BTC ke...
1169,Tlotlanang Bogg,Ke tseo di new developments betsho,148,193,The started this ba sena go bona gore ha golon...,Thanks for the information


In [21]:
FB_Group_Data.tail(40)

,Author,Post Content,No of Reactions,No of Comments,Comment One,Comment Two
1131,Kago Morenaa,"good morning,ke bona matlhotlha pelo aoo,l did...",15,25,Nna sale ke phuaganya ke na le 3 months orange...,Bae Number 1 because of you
1132,Lebo Fifi Matengu,None,3,4,Mascom ke basic mode ya eng phakela o kana kan...,Kopa Le thuse Mascom sale ke emetse call ya ag...
1133,Mayson Thapelo,Nothing pisses off like getting into a store o...,56,34,Nnyaa mme oba togetse na kile ba leka meleko e...,I remember the other at a Chinese shop that is...
1134,Goitsemang Nanz,None,24,20,Bathong mascom nkitsheng mo basic mode ke subs...,Bathong mascom nkitsheng mo basic mode ke subs...
1135,MsKay Mompati,Bathong FNB Botswana kindly explain to me ka c...,36,36,"Nna a me a nna a le mmogo,e ncharger ga 1 fela",Bathong a jaanong lo changile charge ke 377
1136,Bakang Becky Retshabile,I would like to name and shame Peo Finance at ...,6,None,None,None
1137,Sitha Sitha,"Big up to The Management Team( Choppies, Good...",9,None,None,None
1138,Zicky Zakes,"Hi all,Can someone kindly confirm if the renew...",8,11,None,None
1139,Princess Raletsatsi,Mascom why le nkgaola mme k drile se setshwane...,11,4,Nna e mpeile hela mo basic mode mme I subscribed,None
1140,Ramoduba Moreri,Every Customer Complaint. Molepolole Sprinter,191,135,None,None


In [20]:
FB_Group_Data.to_csv('Latest_Dataset_v2.csv', encoding='utf-8', index = True)